# Projeto pinguins

### Bibliotecas

In [2]:
import pandas as pd
import numpy as np

### Abrindo o arquivo

In [4]:
df = pd.read_csv('penguins.csv')
display(df)

,Unnamed: 0,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,4,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,341,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,342,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,343,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


### Análise inicial

In [7]:
# Removendo a primeira coluna
df = df.iloc[:,1:]
display(df)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


In [8]:
# Informações gerais
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
 7   year               344 non-null    int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 21.6+ KB


In [10]:
# Selecionando linhas com dados faltantes
valores_na = df['sex'].isna()
df_nulos = df[valores_na]
display(df_nulos)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN,2007
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN,2007
10,Adelie,Torgersen,37.8,17.1,186.0,3300.0,NaN,2007
11,Adelie,Torgersen,37.8,17.3,180.0,3700.0,NaN,2007
47,Adelie,Dream,37.5,18.9,179.0,2975.0,NaN,2007
178,Gentoo,Biscoe,44.5,14.3,216.0,4100.0,NaN,2007
218,Gentoo,Biscoe,46.2,14.4,214.0,4650.0,NaN,2008
256,Gentoo,Biscoe,47.3,13.8,216.0,4725.0,NaN,2009
268,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,NaN,2009


Podemos observar que apenas 2 pinguins possuem dados faltantes nas colunas bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g e sex (ids 3 e 271). \
Os outros possuem dados faltantes apenas na coluna sex (ids 8, 9, 10, 11, 47, 178, 218, 256 e 268).

### Tratamento de valores nulos

Para tratar os valores faltantes numéricos (bill_length_mm, bill_depth_mm, flipper_length_mm e body_mass_g) decidiu-se preenchê-los com os valores médios dos pinguins da mesma ilha. Como eram apenas 2 pinguins, não foi feita a diferenciação entre machos e fêmeas - até porque não sabemos os sexos desses dois pinguins.

In [23]:
# Substituindo os valores nulos pelos valores médios dos pinguins daquela ilha
df = df.fillna(df.loc[df['island']=='Torgersen',:].mean()).round(2)
df = df.fillna(df.loc[df['island']=='Biscoe',:].mean()).round(2)
display(df)

C:\Users\User\AppData\Local\Temp\ipykernel_8288\2333935484.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.loc[df['island']=='Torgersen',:].mean()).round(2)
C:\Users\User\AppData\Local\Temp\ipykernel_8288\2333935484.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.loc[df['island']=='Biscoe',:].mean()).round(2)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.10,18.70,181.0,3750.00,male,2007
1,Adelie,Torgersen,39.50,17.40,186.0,3800.00,female,2007
2,Adelie,Torgersen,40.30,18.00,195.0,3250.00,female,2007
3,Adelie,Torgersen,38.95,18.43,191.2,3706.37,NaN,2007
4,Adelie,Torgersen,36.70,19.30,193.0,3450.00,female,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.80,19.80,207.0,4000.00,male,2009
340,Chinstrap,Dream,43.50,18.10,202.0,3400.00,female,2009
341,Chinstrap,Dream,49.60,18.20,193.0,3775.00,male,2009
342,Chinstrap,Dream,50.80,19.00,210.0,4100.00,male,2009
